In [103]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

In [119]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = openai_api_key
)

In [120]:
model = "gpt-4o-mini"

In [121]:
system_prompt = "You are a chatbot"

In [122]:
chat_completion = client.chat.completions.create(
    model = model,
    messages = [
        {"role" : "system", "content" : system_prompt},
        {"role" : "user" , "content" : "What is my name"},
    ]
)

chat_completion.choices[0].message.content

"I don't know your name. If you'd like to share it or ask anything else, feel free!"

### ADDING Memory to the Context

In [123]:
agent_memory = {"human" : "Name : Alex"}
system_prompt = "You are a chatbot. You have a section of context called [MEMORY] that contains information relavant to your conversation"

In [124]:
import json

chat_completion = client.chat.completions.create(
    model = model,
    messages = [
        {"role" : "system", "content" : system_prompt + "[MEMORY]\n" + json.dumps(agent_memory)},
        {"role" : "user" , "content" : "What is my name"},
    ]
)
chat_completion.choices[0].message.content

'Your name is Alex.'

### Modifying memory with tools

#### Defining a memory editing tool

In [125]:
agent_memory = {"human" : "", "agent" : ""}

def core_memory_save(section: str, memory: str):
    agent_memory[section] += '\n'
    agent_memory[section] += memory


In [126]:
agent_memory

{'human': '', 'agent': ''}

In [127]:
core_memory_save("human", "My name is Brandon")

In [128]:
agent_memory

{'human': '\nMy name is Brandon', 'agent': ''}

In [129]:
core_memory_description = "Save important information about you, the agent or the human you are chatting with"

core_memory_save_properties = {
    "section" : {
        "type" : "string",
        "enum" : ["human", "agent"],
        "description" : "Must be either 'human'(to save information about human) or 'agent' (to save information about yourself)",
    },
    "memory":{
        "type" : "string",
        "description" : "Memory to save in the section"
    }
}

core_memory_save_metadata = {
    "type" : "function",
    "function": {
        "name" : "core_memory_save",
        "description" : core_memory_description,
        "parameters" : {
            "type" : "object",
            "properties" : core_memory_save_properties,
            "required" : ["section", "memory"]
        }
    }
}


In [130]:
agent_memory = {"human" : ""}
system_prompt = "You are a chatbot. You have a section of context called [MEMORY] that contains informationrelevant to your conversation."
chat_completion = client.chat.completions.create(
    model = model,
    messages = [
        {"role" : "system", "content" : system_prompt},
        {"role" : "system", "content" : "[MEMORY]\n" + json.dumps(agent_memory)},
        {"role" : "user", "content" : "My name is Jiake"},
    ],
    tools = [core_memory_save_metadata]
)

response =  chat_completion.choices[0]
response

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kkkFB0O2XYe6oRnjXHWOh2Za', function=Function(arguments='{"section":"human","memory":"Name is Jiake"}', name='core_memory_save'), type='function')]))

In [131]:
agent_memory

{'human': ''}

In [132]:

arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

{'section': 'human', 'memory': 'Name is Jiake'}

In [133]:
core_memory_save(**arguments)

In [134]:
agent_memory

{'human': '\nName is Jiake'}

#### Running the next agent

In [135]:
chat_completion = client.chat.completions.create(
    model = model,
    messages = [
        {"role" : "system", "content" : system_prompt},
        {"role": "system", "content" : "[MEMORY]\n" + json.dumps(agent_memory)},
        {"role" : "user", "content" : "what is my name ?"},
    ],
    tools = [core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your name is Jiake.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))

### Implementing the agentic loop

In [137]:
agent_memory = {"human" : ""}

In [139]:
system_prompt_os = system_prompt + "You must either call a tool (core_memory_save) or write a response to the user." \
+ "Do not take the same action multiple times!" + "When you learn new information, make sure to always call the core_memory_save tool."

In [141]:
def agent_step(user_message):
    messages = [
        {"role" : "system", "content" : system_prompt_os},
        #memory
        {
            "role" : "system",
            "content" : "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ]
    messages.append({"role" : "user", "content" : user_message})
    #agentic loop
    while True:
        chat_completion = client.chat.completions.create(
            model = model,
            messages = messages,
            tools = [core_memory_save_metadata],
        )
        response = chat_completion.choices[0]

        messages.append(response.message)
        #if not calling a tool(responsing to the user), return
        if not response.message.tool_calls:
            return response.message.content

        #if calling a tool, execute the tool
        else:
            print("TOOL CALL:", response.message.tool_calls[0].function)

            arguments = json.loads(response.message.tool_calls[0].function.arguments)

            core_memory_save(**arguments)
            messages.append({
                "role" : "tool",
                "tool_call_id" : response.message.tool_calls[0].id,
                "name" : "core_memory_save",
                "content" : f"Updated_memory: {json.dumps(agent_memory)}"
            })



In [151]:
agent_step("My favorite color is Blue")
agent_step("My favorite cuisine is Italian")
agent_step("My favorite animal is Dog")

TOOL CALL: Function(arguments='{"section":"human","memory":"Anish\'s favorite animal is Dog."}', name='core_memory_save')


'Got it! Your favorite animal is a dog.'

In [ ]:
chat_completion = client.chat.completions.create(
    model = model,
    messages = [
        {"role" : "system", "content" : system_prompt},
        {"role": "system", "content" : "[MEMORY]\n" + json.dumps(agent_memory)},
        {"role" : "user", "content" : "what is my nfavorite animal ?"},
    ],
    tools = [core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your favorite animal is a dog!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))